In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("data/103_classes_filtered.csv")

In [3]:
X = df['title']
y = df['categories']

In [4]:
X.shape

(381803,)

In [5]:
y[0]

"['cs.NE', 'cs.AI']"

In [6]:
from sklearn.preprocessing import MultiLabelBinarizer
from ast import literal_eval
y = [literal_eval(i) for i in y]
mlb = MultiLabelBinarizer()
y_binarized = mlb.fit_transform(y)
mlb.classes_

array(['astro-ph.IM', 'cond-mat.dis-nn', 'cond-mat.mtrl-sci',
       'cond-mat.stat-mech', 'cs.AI', 'cs.AR', 'cs.CC', 'cs.CE', 'cs.CG',
       'cs.CL', 'cs.CR', 'cs.CV', 'cs.CY', 'cs.DB', 'cs.DC', 'cs.DL',
       'cs.DM', 'cs.DS', 'cs.ET', 'cs.FL', 'cs.GL', 'cs.GR', 'cs.GT',
       'cs.HC', 'cs.IR', 'cs.IT', 'cs.LG', 'cs.LO', 'cs.MA', 'cs.MM',
       'cs.MS', 'cs.NA', 'cs.NE', 'cs.NI', 'cs.OH', 'cs.OS', 'cs.PF',
       'cs.PL', 'cs.RO', 'cs.SC', 'cs.SD', 'cs.SE', 'cs.SI', 'cs.SY',
       'econ.EM', 'econ.GN', 'econ.TH', 'eess.AS', 'eess.IV', 'eess.SP',
       'eess.SY', 'hep-ex', 'math-ph', 'math.AC', 'math.AG', 'math.AP',
       'math.AT', 'math.CA', 'math.CO', 'math.CT', 'math.DG', 'math.DS',
       'math.FA', 'math.GR', 'math.IT', 'math.LO', 'math.MG', 'math.MP',
       'math.NA', 'math.NT', 'math.OC', 'math.PR', 'math.RA', 'math.RT',
       'math.ST', 'nlin.AO', 'nlin.CD', 'nlin.CG', 'physics.ao-ph',
       'physics.app-ph', 'physics.bio-ph', 'physics.chem-ph',
       'physics.comp

In [7]:
import pickle
with open('./data/y_binarized_data.pkl','wb') as f:
    pickle.dump(y_binarized,f)

In [8]:
y_binarized

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [9]:
# class distribution
class_sum = np.sum(y_binarized, axis =0)
class_distribution = pd.DataFrame(columns=['class','count'])
for i in range(len(mlb.classes_)):
    class_distribution.loc[i] = [mlb.classes_[i],class_sum[i]]
print(class_distribution)

                  class  count
0           astro-ph.IM    212
1       cond-mat.dis-nn    367
2     cond-mat.mtrl-sci    179
3    cond-mat.stat-mech    423
4                 cs.AI  51093
..                  ...    ...
98              stat.AP   1636
99              stat.CO    514
100             stat.ME    786
101             stat.ML  34643
102             stat.TH   1348

[103 rows x 2 columns]


In [10]:
class_distribution_sorted = class_distribution.sort_values('count', ascending=False)

In [11]:
plt.figure(figsize=(300, 50), dpi=100)
plt.xticks(fontsize=14)
plt.bar(class_distribution_sorted['class'], class_distribution_sorted['count'])
#normalise with num of samples

<BarContainer object of 103 artists>

In [12]:
X.shape

(381803,)

title preprocessing

In [13]:
from nltk.corpus import stopwords
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/siddhipotdar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
# tokenize words and remove stopwords

stopwords = nltk.corpus.stopwords.words('english')
tokenizer = nltk.WordPunctTokenizer()

def preprocess(text: str) -> str:
    text = tokenizer.tokenize(text.lower().strip())
    text = [ x for x in text if x not in stopwords]
    return ' '.join(text)



In [15]:
X_token = []
for item in X:
    item = re.sub(r'[^a-zA-Z\s]','', item, re.I)
    X_token.append(preprocess(item))
    #print(item)
    

In [16]:
X_token = np.array(X_token)

In [28]:
np.save('./data/X_token', X_token)

In [17]:
# !wget https://github.com/facebookresearch/fastText/archive/v0.9.2.zip
# !unzip v0.9.2.zip
# !cd fastText-0.9.2
# !pip install ./fastText-0.9.2/.


Finding fastttext sentence embeddings

In [18]:
import fasttext
import fasttext.util

#fasttext.util.download_model('en', if_exists='ignore')
model = fasttext.load_model('./fastText-0.9.2/cc.en.300.bin')
    

In [19]:
# make a dictionary, store repeating words from the dataset inside it

embedding_map = dict()
for sentence in X_token:
    for word in sentence.split():
        if word not in embedding_map:
            embedding_map[word] = model.get_word_vector(word)

In [20]:
#print(embedding_map)

In [21]:
def get_fasttext_embeddings(x_data, y_data):
    sentence_embedding = []
    sample = 0
    for sentence in x_data:
        word_embedding = np.zeros((300,))
        count = 0
        for word in sentence.split():
            word_embedding+=embedding_map[word]
            count += 1
        if count != 0:
            sentence_embedding.append(word_embedding/count)
        else:
            sentence_embedding.append(word_embedding)
        sample+=1
    return np.array(sentence_embedding), y_data


In [22]:
X_embeddings, y_label = get_fasttext_embeddings(X_token, y_binarized)

In [23]:
len(X_embeddings)
print(X_embeddings.shape)
len(y_label)


(381803, 300)


381803

In [24]:
np.save('./data/final_x_103_50pct_os', X_embeddings)
np.save('./data/final_train_y_103_50pct_os', y_label)

In [25]:
from sklearn.feature_extraction.text  import TfidfVectorizer
def get_tfidf_embeddings(x_data, y_data):
  tf_idf_vect = TfidfVectorizer(stop_words=None)
  tf_idf_vect.fit(x_data)
  final_tf_idf = tf_idf_vect.transform(x_data)
  tfidf_feat = tf_idf_vect.get_feature_names()
  return tfidf_feat, final_tf_idf, y_data


In [26]:
tfidf_feat, X_tfidf, y_label = get_tfidf_embeddings(X_token, y_binarized)

/Users/siddhipotdar/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [27]:
from scipy.sparse import hstack, vstack

print(X_tfidf.shape)
print(X_tfidf[0])
print(X_embeddings.shape)
#from sklearn.pipeline import FeatureUnion

#X_union = FeatureUnion([("tfidf", X_tfidf), ("fasttext", X_embeddings)])
X_tfidf_embedding = hstack((X_tfidf, X_embeddings))
print(X_tfidf_embedding.shape)



np.save('./data/X_fasttext_embeddings', X_embeddings)
np.save('./data/X_tfidf_embeddings', X_tfidf)
np.save('./data/X_tfidf_and_fasttext_embeddings', X_tfidf_embedding)
np.save('./data/y_embeddings', y_label)
np.save('./data/y_tags',mlb.classes_)



(381803, 127209)
  (0, 106337)	0.33102962498570715
  (0, 103800)	0.42046434313628245
  (0, 82077)	0.3283122393721111
  (0, 62766)	0.3254470543645825
  (0, 55028)	0.29321285148098913
  (0, 34836)	0.471377330665021
  (0, 1522)	0.27950590870022085
  (0, 1356)	0.33709122604994246
(381803, 300)
(381803, 127509)
